# USA Housing Model Notebook 
#### In this notebook we will create, train , validate, and do prediction

<h1>Table of contents</h1>
​
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#explore_dataset">Explore dataset</a></li>
        <li><a href="#create">Create table and import data</a></li>
        <li><a href="#cm">Create Model</a></li>
        <li><a href="#tm">Train Model</a></li>
        <li><a href="#vm">Validate Model</a></li>
        <li><a href="#pr">Do Prediction</a></li>   
        <li><a href="#sm">Summary</a></li>  
    </ol>
</div>
<br>
<hr>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

<div id="explore_dataset">
    <h1>Explore dataset</h1>   
</div>

In [2]:
USAhousing = pd.read_csv('./datasets/USA_Housing_train.csv')
USAhousing.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [15]:
USAhousing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Avg. Area Income              4000 non-null   float64
 1   Avg. Area House Age           4000 non-null   float64
 2   Avg. Area Number of Rooms     4000 non-null   float64
 3   Avg. Area Number of Bedrooms  4000 non-null   float64
 4   Area Population               4000 non-null   float64
 5   Price                         4000 non-null   float64
dtypes: float64(6)
memory usage: 187.6 KB


In [16]:
USAhousing.describe()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4.000000e+03
mean,68561.764989,5.982444,6.989255,3.974775,36279.220348,1.235349e+06
std,10693.493261,0.990669,1.004391,1.231435,9899.991119,3.544019e+05
min,17796.631190,2.644304,3.236194,2.000000,172.610686,1.593866e+04
25%,61474.294678,5.334776,6.308615,3.140000,29550.400770,9.997036e+05
50%,68655.504150,5.969464,7.001368,4.045000,36240.428580,1.237303e+06
75%,75781.049625,6.663936,7.665612,4.490000,43000.807275,1.469933e+06
max,107701.748400,9.519088,10.759588,6.500000,69592.040240,2.469066e+06


In [3]:
import  utility

connection = utility.get_db_connection()

cur = connection.cursor()

<div id="create">
    <h1>Create table and import data</h1>   
</div>

In [3]:
table = "usa_housing_train"
cur.execute(f"CREATE TABLE {table} ( id INT(32) AUTO_INCREMENT PRIMARY KEY, AvgAreaIncome FLOAT, AvgAreaHouseAge FLOAT,AvgAreaRooms FLOAT,AvgAreaBedRooms FLOAT,AreaPopulation FLOAT,Price FLOAT)")
table = "usa_housing_validate"
cur.execute(f"CREATE TABLE {table} ( id INT(32) AUTO_INCREMENT PRIMARY KEY, AvgAreaIncome FLOAT, AvgAreaHouseAge FLOAT,AvgAreaRooms FLOAT,AvgAreaBedRooms FLOAT,AreaPopulation FLOAT,Price FLOAT)")

NameError: name 'cur' is not defined

## Import data

In [ ]:
USAhousingTrain = pd.read_csv('./datasets/USA_Housing_train.csv')
USAhousingTrain.head()

In [ ]:
#print(type(USAhousingTrain))
table = "usa_housing_train"  
cur.execute(f"TRUNCATE TABLE {table}")
for i in range(len(USAhousingTrain)):
    #print(USAhousingTrain.iloc[i, 0], USAhousingTrain.iloc[i, 1])
    cur.execute(f"INSERT INTO {table} (AvgAreaIncome, AvgAreaHouseAge, AvgAreaRooms, AvgAreaBedRooms, AreaPopulation, Price) VALUES ({USAhousingTrain.iloc[i, 0]}, {USAhousingTrain.iloc[i, 1]}, {USAhousingTrain.iloc[i, 2]} , {USAhousingTrain.iloc[i, 3]}, {USAhousingTrain.iloc[i, 4]}, {USAhousingTrain.iloc[i, 5]})")

connection.commit()

#print('Values inserted')    

In [ ]:
#print(type(USAhousingTrain))
USAhousingVal = pd.read_csv('./datasets/USA_Housing_validate.csv')
USAhousingVal.head()
table = "SQLUser.usa_housing_validate"  
cur.execute(f"TRUNCATE TABLE {table}")
for i in range(len(USAhousingVal)):
    #print(USAhousingTrain.iloc[i, 0], USAhousingTrain.iloc[i, 1])
    cur.execute(f"INSERT INTO {table} (AvgAreaIncome, AvgAreaHouseAge, AvgAreaRooms, AvgAreaBedRooms, AreaPopulation, Price) VALUES ({USAhousingTrain.iloc[i, 0]}, {USAhousingTrain.iloc[i, 1]}, {USAhousingTrain.iloc[i, 2]} , {USAhousingTrain.iloc[i, 3]}, {USAhousingTrain.iloc[i, 4]}, {USAhousingTrain.iloc[i, 5]})")

connection.commit()

<div id="cm">
    <h1>Create Model</h1>   
</div>

In [ ]:
modelName = "USAHousingPriceModel"
dataColumn = "Price"
dataTable= "usa_housing_train"
cur.execute(f"CREATE MODEL {modelName} PREDICTING ({dataColumn})  FROM {dataTable}") 
#Create model asdf predicting (alpha) from DATA.STARS_TRAIN
connection.commit()
print('Model created')

<div id="tm">
    <h1>Train Model</h1>   
</div>

In [ ]:
#### Train model- PLease note it might take time based on number of records
modelName = "USAHousingPriceModel"
dataTable= "usa_housing_train"
cur.execute(f"TRAIN MODEL {modelName} FROM {dataTable}")
connection.commit()
print('Model Trained')

<div id="vm">
    <h1>Validate Model</h1>   
</div>

In [5]:
modelName = "USAHousingPriceModel"
ValidationRunName = "USAHousingPriceValidateuse"
TrainModelValidate="USAHousingPriceModel_t1"
TableToValidate = "usa_housing_validate"
cur.execute(f"Validate model {modelName} As {ValidationRunName} use {TrainModelValidate} from {TableToValidate}")
connection.commit()
print('Model Validated')

NameError: name 'cur' is not defined

#### Printing Validation Metrics

In [1]:
import utility
modelName = "USAHousingPriceModel"
ValidationRunName = "USAHousingPriceValidateuse"
#get metrics values
mse,rmse,var,r2 = utility.get_validation_metrics(modelName,ValidationRunName)
print("Validation Metrics for Regression Models")
print("MSE = "+ mse)
print("RMSE = "+ rmse)
print("Variance = "+ var)
print("R2 = "+ r2)

Validation Metrics for Regression Models
MSE = 5287446379.99
RMSE = 72714.83
Variance = 126980693899.03
R2 = 0.96


When training the model, the accuracy is considered satisfactory if the R2 value is greater than 95%.

<div id="pr">
    <h1>Do Prediction</h1>   
</div>

In [11]:
import  utility
import pandas as pd
connection = utility.get_db_connection()
cur = connection.cursor()
stat = "SELECT TOP(100) PREDICT(USAHousingPriceModel use USAHousingPriceModel_t1) as prediction, Price, * FROM SQLUser.usa_housing_validate"
cur.execute(stat)
data = cur.fetchall()
df = pd.DataFrame (data = data, columns = utility.get_cols(cur.description))
print(df.head())
cur.close()
connection.close()

           prediction         Price  id  AvgAreaIncome  AvgAreaHouseAge   
0   1165769.756251161  1.059034e+06   1    79545.45857         5.682861  \
1  1448321.0345289926  1.505891e+06   2    79248.64245         6.002900   
2   1153023.796213299  1.058988e+06   3    61287.06718         5.865890   
3   1143096.669677456  1.260617e+06   4    63345.24005         7.188236   
4   754634.7194245736  6.309435e+05   5    59982.19723         5.040555   

   AvgAreaRooms  AvgAreaBedRooms  AreaPopulation         Price  
0      7.009188             4.09     23086.80050  1.059034e+06  
1      6.730821             3.09     40173.07217  1.505891e+06  
2      8.512727             5.13     36882.15940  1.058988e+06  
3      5.586729             3.26     34310.24283  1.260617e+06  
4      7.839388             4.23     26354.10947  6.309435e+05  


<div id="sm">
    <h1>Summary</h1>   
</div>

In this notebook we covered the following:
- Explore USA Housing dataset
- Created table by using IS Cloud Services and imported the data
- Created Model
- Trained Model
- Validate Model
- Done the Pridiction

Thanks